In [1]:
from dotenv import load_dotenv
load_dotenv()

%load_ext google.cloud.bigquery

In [2]:
%%bigquery open_issues_60_days
SELECT
   opened.issues_id
    , opened.stars_repo_id
    , opened.actor_login 
    , opened.html_url
    , opened.opened_created_at
    , closed.closed_created_at
    , TIMESTAMP_DIFF(closed.closed_created_at, opened.opened_created_at, DAY) as length_in_days
    , opened.repo_name
    , opened.stars
    , opened.labels
    , opened.title
    , best_actor_for_repo.actor_login as best_solver
    , DATE_DIFF(CURRENT_DATE(), DATE(opened.opened_created_at), DAY) as days_outstanding
FROM (
  SELECT
      gd.id as issues_id
    , rs.repo_id stars_repo_id
    , payload
    , JSON_EXTRACT_SCALAR(payload, "$.issue.html_url") as html_url
    , LOWER(JSON_EXTRACT_SCALAR(payload, '$.issue.labels[0].name')) as labels
    , LOWER(JSON_EXTRACT_SCALAR(payload, '$.issue.title')) as title
    , repo.name as repo_name
    , repo.url
    , actor.login actor_login
    , created_at as opened_created_at
    , stars
  FROM
    `githubarchive.day.201*` gd
  LEFT JOIN 
    `fh-bigquery.github_extracts.repo_stars` rs ON gd.repo.name = rs.repo_name
  WHERE
    type = 'IssuesEvent'
  AND
    JSON_EXTRACT_SCALAR(payload, "$.action") = 'opened'
) opened
LEFT JOIN
(
  SELECT
    gd.id as id
    , JSON_EXTRACT_SCALAR(payload, "$.issue.html_url") as html_url
    , repo.url
    , actor.login as actor_login
    , created_at as closed_created_at
  FROM
  `githubarchive.day.201*` gd
  WHERE
    type = 'IssuesEvent'
  AND
    JSON_EXTRACT_SCALAR(payload, "$.action") = 'closed'
) closed
ON opened.html_url = closed.html_url AND opened.url = closed.url
JOIN (
   SELECT 
      e.*
   FROM
   (
      SELECT
        c.repo_url
      , c.repo_name
      , MAX(c.number_of_issues_closed_by_actor) as number_of_issues_closed_by_actor
      FROM
      (
       SELECT
       repo.url repo_url
        , repo.name as repo_name
        , COUNT(created_at) as number_of_issues_closed_by_actor
       FROM
       `githubarchive.day.201*` gd
       WHERE
       type = 'IssuesEvent'
       AND
       payload LIKE '%"action":"closed"%'
       GROUP BY
       repo.url, repo.name
      ) c
      GROUP BY c.repo_url, c.repo_name
  ) d
  JOIN
    (
      SELECT
      actor.login as actor_login
      , repo.url repo_url
      , repo.name as repo_name
      , COUNT(created_at) as number_of_issues_closed_by_actor
     FROM
      `githubarchive.day.201*` gd
     WHERE
      type = 'IssuesEvent'
     AND
     payload LIKE '%"action":"closed"%'
     GROUP BY actor_login, repo_url, repo_name
    ) e
  ON d.repo_url = e.repo_url AND d.repo_name = e.repo_name and d.number_of_issues_closed_by_actor = e.number_of_issues_closed_by_actor
) best_actor_for_repo
ON opened.repo_name = best_actor_for_repo.repo_name
WHERE 
  opened_created_at < TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 60 DAY) 
  AND opened_created_at > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 61 DAY)
  AND closed_created_at IS NULL
  AND opened.opened_created_at is not null
ORDER BY days_outstanding asc, stars desc
LIMIT 16000

,issues_id,stars_repo_id,actor_login,html_url,opened_created_at,closed_created_at,length_in_days,repo_name,stars,labels,title,best_solver,days_outstanding
0,8775657555,98695067.0,sirpalee,https://github.com/LumaPictures/usd-arnold/iss...,2018-12-19 00:07:20+00:00,None,None,LumaPictures/usd-arnold,11.0,None,handle aovs and redesign data passed from the ...,sirpalee,60
1,8775678038,56768117.0,danil179,https://github.com/LeagueSandbox/ReplayInspect...,2018-12-19 00:13:25+00:00,None,None,LeagueSandbox/ReplayInspector,10.0,None,adding parsing stage,danil179,60
2,8775651419,1037499.0,4rknova,https://github.com/4rknova/xtracer/issues/46,2018-12-19 00:05:33+00:00,None,None,4rknova/xtracer,9.0,None,refactor uv integrator,4rknova,60
3,8775642090,111564005.0,KaiKikuchi,https://github.com/KaiKikuchi/ServerRedirect/i...,2018-12-19 00:02:55+00:00,None,None,KaiKikuchi/ServerRedirect,2.0,None,incompatible with spigot 1.13.2,KaiKikuchi,60
4,8775679228,106573721.0,yangwenmai,https://github.com/yangwenmai/reading/issues/870,2018-12-19 00:13:47+00:00,None,None,yangwenmai/reading,1.0,None,十年磨剑大巧不工,yangwenmai,60
5,8775665134,NaN,critmcdonald,https://github.com/utdata/icj-project-template...,2018-12-19 00:09:38+00:00,None,None,utdata/icj-project-template,NaN,None,google docs/sheets support,critmcdonald,60
6,8775669384,NaN,MrToph,https://github.com/ipfs-shipyard/ipscend/issue...,2018-12-19 00:10:53+00:00,None,None,ipfs-shipyard/ipscend,NaN,None,cannot ipfs start,fiatjaf,60
7,8775660183,NaN,Mouleh,https://github.com/Mouleh/SBT/issues/2,2018-12-19 00:08:09+00:00,None,None,Mouleh/SBT,NaN,None,script bucheron,Mouleh,60
8,8775678623,NaN,birdsarah,https://github.com/mozilla/Overscripted-Data-A...,2018-12-19 00:13:36+00:00,None,None,mozilla/Overscripted-Data-Analysis-Challenge,NaN,None,updating previous fingerprinting / tracking su...,mlopatka,60
9,8775669647,NaN,philiprbrenan,https://github.com/philiprbrenan/notifications...,2018-12-19 00:10:58+00:00,None,None,philiprbrenan/notifications,NaN,None,word to dita plex5 - title and file report com...,philiprbrenan,60


In [3]:
import requests
import json
import os

GITHUB_URL = "https://api.github.com/"

"""
Function to get JSON response from a URL
:params:
    url     string
:return:
            JSON
"""
def __get_json_response(url):
    headers = {'Authorization': 'token ' + os.environ['GITHUB_API_KEY']}
    response = requests.get(url, headers=headers)
    return json.loads(response.text)

"""
Function to add email to a set of emails and set a loop break flag
:params:
    email_set       set         set of all the emails for the user
    email           string      new email to be added
    max_len         integer     maximum number of emails to be extracted
:return:
    email_set       set         set of all the emails for the user
    break_flag      boolean     if max_limit is reached, break_flag is set to True
"""
def __add_email(email_set, email, max_len):
    email_set = email_set | set([email])
    break_flag = (len(email_set) >= max_len)

    return email_set, break_flag

"""
Function to get user emails using GitHub APIs
:params:
    user        string      a valid GitHub username
    max_limit   integer     maximum number of email ID to be fetched
:return:
    user_email  set         a set of all emails extracted
    message     string      if any error occurs, this holds the respective error message
"""
def __get_github_emails(user, max_limit):
    user_email = set([])
    break_flag = False
    try:
        users_profile_url = GITHUB_URL + "users/{0}".format(user)
        response = __get_json_response(users_profile_url)

        # some error encountered
        if 'message' in response:
            if response['message'] == 'Not Found':
                return u'You need to enter a valid GitHub Username'
            else:
                return response['message']
        name = response['name']
        user_name = response['name']
        company = response['company']
        # if user has a public email, add that to the set of emails
        if response['email']:
            user_email, break_flag = __add_email(user_email, response['email'], max_limit)

        if not break_flag:
            users_repository_url = GITHUB_URL + "users/{0}/repos?type=owner&sort=updated".format(user)
            response = __get_json_response(users_repository_url)

            for repo in response:
                if not repo['fork']:
                    users_repository_name = repo['full_name']
                    repos_commit_url = GITHUB_URL + "repos/{0}/commits".format(users_repository_name)
                    commit_reponse = __get_json_response(repos_commit_url)
                    
                    possible_positions = ['committer', 'author']

                    for commit in commit_reponse:
                        for i in possible_positions:
                            if commit['commit'][i]['name'] == user_name:
                                email_string = commit['commit'][i]['email']
                                if "noreply" not in email_string:
                                    user_email, break_flag = __add_email(user_email, email_string, max_limit)

                        if break_flag:
                            break

                if break_flag:
                    break

        if len(user_email) > 0:
            return user_email, name, company
        else:
            return u'No emails found', u'No first name found', u'No company found'

    except requests.exceptions.ConnectionError:
        return u'Proper internet connection not found', u'Proper internet connection not found', u'Proper internet connection not found'

"""
Function to get the emails associated to a username on GitHub
:params:
    username    string      a valid GitHub username
    num         integer     maximum number of email ID to be fetched, default 1
:return:
    response    JSON response
        success     boolean     flag to determine other key in JSON
        email       list        if 'success' is True, list of all the emails fetched
        message     string      if 'success' is False, returns the error message
"""
def get(username, num=1):
    email, name, co = __get_github_emails(username, num)
    if type(email) == set:
        response = {
            'success' : True,
            'data': [email.pop(), name, co]
        }
    else:
        response = {
            'success' : False,
            'message' : [email, name, co]
        }

    return response

In [4]:
import pandas as pd
result = pd.DataFrame()
for index, row in open_issues_60_days.iterrows():
    print(index)
    issues_id = row['issues_id']
    stars_repo_id = row['stars_repo_id']
    actor_login = row['actor_login']
    html_url = row['html_url']
    opened_created_at = row['opened_created_at']
    closed_created_at = row['closed_created_at']
    length_in_days = row['length_in_days']
    repo_name = row['repo_name']
    stars = row['stars']
    labels = row['labels']
    best_solver = row['best_solver']
    days_outstanding = row['days_outstanding']
    title = row['title'].capitalize()
    try:
        resp = get(actor_login)
        if 'data' in resp:
            email, name, co  =resp['data']
            row['email'] = email
            row['name'] = name
            row['company'] = co
            if ".com" in email:
                result = result.append(row, ignore_index=True)
                print(resp['data'])
                print(result.shape)
    except Exception as e:
        print(e)
    


0
[u'sirpalee@gmail.com', u'Pal Mezei', u'@LumaPictures ']
(1, 16)
1
2
[u'nikos.papadopoulos@bisimulations.com', u'Nikos Papadopoulos', None]
(2, 16)
3
[u'kai.kikuchi27@gmail.com', u'Kai', None]
(3, 16)
4
[u'yangwen.yw@gmail.com', u'maiyang', None]
(4, 16)
5
6
[u'christoph.michel.1992@gmail.com', u'Christoph Michel', None]
(5, 16)
7
8
[u'sarah@bonvaya.com', u'Sarah Bird', u'@mozilla']
(6, 16)
9
[u'philiprbrenan@gmail.com', u'philip r brenan', u'Appa Apps Ltd Inc']
(7, 16)
10
11
[u'thelevybreaks@gmail.com', u'James Levy', u'@WeLens']
(8, 16)
12
[u'ash.d.wilson@gmail.com', u'Ash Wilson', u'CloudPassage']
(9, 16)
13
string indices must be integers
14
15
16
17
18
19
string indices must be integers
20
[u'qgeissmann@gmail.com', u'Quentin Geissmann', u'University of British Columbia']
(10, 16)
21
[u'qgeissmann@gmail.com', u'Quentin Geissmann', u'University of British Columbia']
(11, 16)
22
23
string indices must be integers
24
[u'subhan.michael@gmail.com', u'Subhan  Michael', None]
(12, 16)
2

[u'a.goyal@outlook.com', u'Anuj Goyal', u'Colorado School of Mines, National Renewable Energy Laboratory']
(105, 16)
253
[u'nicholdav@gmail.com', u'David Nicholson', None]
(106, 16)
254
[u'yussandeveloper@gmail.com', u'Yusuf A.H.', u'@idmore @SebangsaHQ ']
(107, 16)
255
256
[u'tula_post@yahoo.com', u'Tula', None]
(108, 16)
257
[u'hanscees@hanscees.com', u'Hans-Cees Speel', None]
(109, 16)
258
259
string indices must be integers
260
string indices must be integers
261
[u'spencer.bliven@gmail.com', u'Spencer Bliven', u'Paul Scherrer Institute']
(110, 16)
262
string indices must be integers
263
[u'nico.thiebes@xing.com', u'Nico Thiebes', u'@XING']
(111, 16)
264
[u'nico.thiebes@xing.com', u'Nico Thiebes', u'@XING']
(112, 16)
265
[u'aminah.nuraini92@gmail.com', u'Aminah Nuraini', None]
(113, 16)
266
string indices must be integers
267
[u'nico.thiebes@xing.com', u'Nico Thiebes', u'@XING']
(114, 16)
268
string indices must be integers
269
[u'7355233@qq.com', u'wellbye', None]
(115, 16)
270
[u

450
451
[u'christian.cousquer@gmail.com', u'Christian Cousquer', u'Sorbonne Universit\xe9 (formerly called UPMC)']
(207, 16)
452
[u'thejendiamond@gmail.com', u'JenDiamond', u'Shifty']
(208, 16)
453
[u'slianoglou@gmail.com', u'Steve Lianoglou', u'Denali Therapeutics']
(209, 16)
454
[u'jianghui19940917@hotmail.com', u'JiangHui', u'ZheJiang University']
(210, 16)
455
456
457
458
459
[u'zhengzhongzhao@gmail.com', u'Snowmanzzz(Zhengzhong Zhao)', u'Free, Pre SAP Labs China, Microsoft']
(211, 16)
460
461
[u'quinn@quinnweber.com', u'Quinn Weber', u'@convoyinc ']
(212, 16)
462
463
[u'amir.asaraf@gmail.com', u'Amir Asaraf', None]
(213, 16)
464
465
[u'terary@gmail.com', u'Terary Chambers', None]
(214, 16)
466
467
468
469
string indices must be integers
470
[u'chris.foo@gmail.com', u'Christopher Foo', None]
(215, 16)
471
[u'sgodwincs@gmail.com', u'Scott Godwin', u'Google']
(216, 16)
472
[u'fassko@gmail.com', u'Kristaps Grinbergs', u'@Qminder ']
(217, 16)
473
474
475
476
[u'adnanaadi06@gmail.com', 

[u'mhylle@yahoo.com', u'Martin Hylleberg', None]
(310, 16)
708
709
[u'josh.beckwith@gmail.com', u'Josh Beckwith', u'Posit Labs']
(311, 16)
710
711
[u'hello@magnusdahlgren.com', u'Magnus Dahlgren', None]
(312, 16)
712
713
714
715
716
[u'th.boby@gmail.com', u'Thomas Boby', None]
(313, 16)
717
string indices must be integers
718
719
720
[u'522981616@qq.com', u'Omelet', u'@Ringcentral']
(314, 16)
721
[u'romeara@live.com', u"Ryan O'Meara", None]
(315, 16)
722
[u'caseyhoepner@gmail.com', u'Casey Hoepner', None]
(316, 16)
723
724
[u'lifeich0@gmail.com', None, None]
(317, 16)
725
[u'gabe1118@yahoo.com', u'Gabriel Ochoa', u'@Bloomberg']
(318, 16)
726
string indices must be integers
727
728
[u'agiron123@gmail.com', u'Andre Giron', None]
(319, 16)
729
730
[u'esenbekkd@gmail.com', u'Esenbek Kydyr uulu', u'@horizontalsystems']
(320, 16)
731
732
733
734
735
[u'bennnowak@gmail.com', u'Ben Nowak', u'@tyjsoftware @tyjeveryday @coding-boot-camp']
(321, 16)
736
737
738
[u'zayd62@gmail.com', u'Zayd Patel'

string indices must be integers
917
918
919
920
[u'alex.ginns@thirstystudios.com.au', u'Alex Ginns', u'Thirsty Studios']
(408, 16)
921
string indices must be integers
922
923
924
[u'agi.novanta@gmail.com', u'Agi Sferro', None]
(409, 16)
925
926
[u'mijiang@microsoft.com', u'Miao Jiang', u'Microsoft']
(410, 16)
927
string indices must be integers
928
929
930
[u'qcthreestones@gmail.com', u'Chen Qiu', u'BUPT']
(411, 16)
931
932
[u'jeffygisgreat@gmail.com', u'JadeYang(\u6768\u743c\u749e)', u'NICE LINKS']
(412, 16)
933
[u'yuxinwang9999@gmail.com', u'Yuxin Wang', None]
(413, 16)
934
[u'hello@magnusdahlgren.com', u'Magnus Dahlgren', None]
(414, 16)
935
[u'f89647779@gmail.com', u'Hung Yeh Lin', None]
(415, 16)
936
[u'lyalinstas@gmail.com', u'Stanislav Lyalin', u'Home']
(416, 16)
937
938
939
[u'moshfeu@gmail.com', u'Mosh Feu', u'@wix ']
(417, 16)
940
[u'thisaru@wso2.com', u'Thisaru Guruge', u'@WSO2 @ballerina-lang @ballerina-platform @ballerinax ']
(418, 16)
941
942
[u'email@dylanfareed.com', u'

[u'kgr.ze21ro@gmail.com', u'yu-kgr', None]
(499, 16)
1158
1159
string indices must be integers
1160
1161
1162
string indices must be integers
1163
1164
1165
1166
1167
1168
1169
1170
[u'tomkeays@gmail.com', u'Tom Keays', None]
(500, 16)
1171
1172
1173
1174
1175
1176
1177
[u'mijiang@microsoft.com', u'Miao Jiang', u'Microsoft']
(501, 16)
1178
[u'mijiang@microsoft.com', u'Miao Jiang', u'Microsoft']
(502, 16)
1179
1180
1181
string indices must be integers
1182
1183
1184
1185
1186
1187
1188
[u'mijiang@microsoft.com', u'Miao Jiang', u'Microsoft']
(503, 16)
1189
[u'raj.swapnil99@gmail.com', u'Swapnil Raj', None]
(504, 16)
1190
1191
1192
1193
1194
string indices must be integers
1195
[u'haymanjoyce@gmail.com', u'Richard Hayman-Joyce', u'Lemonsunday Ltd']
(505, 16)
1196
1197
1198
[u'oldes.huhuman@gmail.com', u'Oldes Huhuman', u'Freelancer']
(506, 16)
1199
1200
[u'tdiesler@redhat.com', u'Thomas Diesler', u'RedHat Inc.']
(507, 16)
1201
1202
[u'albus.marcel@gmail.com', u'Marcel Albus', None]
(508, 

1433
1434
1435
1436
1437
1438
1439
1440
1441
1442
1443
1444
1445
1446
1447
1448
1449
1450
1451
1452
1453
1454
1455
1456
1457
1458
1459
1460
1461
1462
1463
1464
1465
1466
1467
1468
1469
1470
1471
1472
1473
1474
1475
1476
1477
1478
1479
1480
1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494
1495
1496
1497
1498
1499
1500
1501
1502
1503
1504
1505
1506
1507
1508
1509
1510
1511
1512
1513
1514
1515
1516
1517
1518
1519
1520
1521
1522
1523
1524
1525
1526
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
1545
1546
1547
1548
1549
1550
1551
1552
1553
1554
1555
1556
1557
1558
1559
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
1571
1572
1573
1574
1575
1576
1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600
1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632


3084
3085
3086
3087
3088
3089
3090
3091
3092
3093
3094
3095
3096
3097
3098
3099
3100
3101
3102
3103
3104
3105
3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
3116
3117
3118
3119
3120
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3141
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3176
3177
3178
3179
3180
3181
3182
3183
3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3208
3209
3210
3211
3212
3213
3214
3215
3216
3217
3218
3219
3220
3221
3222
3223
3224
3225
3226
3227
3228
3229
3230
3231
3232
3233
3234
3235
3236
3237
3238
3239
3240
3241
3242
3243
3244
3245
3246
3247
3248
3249
3250
3251
3252
3253
3254
3255
3256
3257
3258
3259
3260
3261
3262
3263
3264
3265
3266
3267
3268
3269
3270
3271
3272
3273
3274
3275
3276
3277
3278
3279
3280
3281
3282
3283


3680
3681
[u'joni7777@gmail.com', u'Jonathan Erel', None]
(656, 16)
3682
string indices must be integers
3683
3684
[u'jfrey@redhat.com', u'Jason Frey', u'Red Hat - @RedHatOfficial']
(657, 16)
3685
3686
3687
string indices must be integers
3688
3689
3690
3691
[u'abethcrane@gmail.com', u'Beth Crane', u'Microsoft']
(658, 16)
3692
3693
3694
[u'james@hendercine.com', u'James Henderson', u'Hendercine Productions']
(659, 16)
3695
[u'tdiesler@redhat.com', u'Thomas Diesler', u'RedHat Inc.']
(660, 16)
3696
string indices must be integers
3697
[u'feldman.sean@gmail.com', u'Sean Feldman', u'Highco Consulting LTD']
(661, 16)
3698
string indices must be integers
3699
string indices must be integers
3700
[u'hexagenia@hotmail.com', u'Scott D. Miller', u'Red Skeeter']
(662, 16)
3701
string indices must be integers
3702
string indices must be integers
3703
3704
string indices must be integers
3705
[u'jroland015004@gmail.com', u'Jessica Roland', None]
(663, 16)
3706
[u'mforets@gmail.com', u'Marcelo Foret

In [5]:

result.to_csv('open_issue_60_days.csv', encoding = 'utf-8')

In [6]:
result

,actor_login,best_solver,closed_created_at,company,days_outstanding,email,html_url,issues_id,labels,length_in_days,name,opened_created_at,repo_name,stars,stars_repo_id,title
0,sirpalee,sirpalee,None,@LumaPictures,60.0,sirpalee@gmail.com,https://github.com/LumaPictures/usd-arnold/iss...,8775657555,None,None,Pal Mezei,2018-12-19 00:07:20+00:00,LumaPictures/usd-arnold,11.0,98695067.0,handle aovs and redesign data passed from the ...
1,4rknova,4rknova,None,None,60.0,nikos.papadopoulos@bisimulations.com,https://github.com/4rknova/xtracer/issues/46,8775651419,None,None,Nikos Papadopoulos,2018-12-19 00:05:33+00:00,4rknova/xtracer,9.0,1037499.0,refactor uv integrator
2,KaiKikuchi,KaiKikuchi,None,None,60.0,kai.kikuchi27@gmail.com,https://github.com/KaiKikuchi/ServerRedirect/i...,8775642090,None,None,Kai,2018-12-19 00:02:55+00:00,KaiKikuchi/ServerRedirect,2.0,111564005.0,incompatible with spigot 1.13.2
3,yangwenmai,yangwenmai,None,None,60.0,yangwen.yw@gmail.com,https://github.com/yangwenmai/reading/issues/870,8775679228,None,None,maiyang,2018-12-19 00:13:47+00:00,yangwenmai/reading,1.0,106573721.0,十年磨剑大巧不工
4,MrToph,fiatjaf,None,None,60.0,christoph.michel.1992@gmail.com,https://github.com/ipfs-shipyard/ipscend/issue...,8775669384,None,None,Christoph Michel,2018-12-19 00:10:53+00:00,ipfs-shipyard/ipscend,NaN,NaN,cannot ipfs start
5,birdsarah,mlopatka,None,@mozilla,60.0,sarah@bonvaya.com,https://github.com/mozilla/Overscripted-Data-A...,8775678623,None,None,Sarah Bird,2018-12-19 00:13:36+00:00,mozilla/Overscripted-Data-Analysis-Challenge,NaN,NaN,updating previous fingerprinting / tracking su...
6,philiprbrenan,philiprbrenan,None,Appa Apps Ltd Inc,60.0,philiprbrenan@gmail.com,https://github.com/philiprbrenan/notifications...,8775669647,None,None,philip r brenan,2018-12-19 00:10:58+00:00,philiprbrenan/notifications,NaN,NaN,word to dita plex5 - title and file report com...
7,jamslevy,jamslevy,None,@WeLens,60.0,thelevybreaks@gmail.com,https://github.com/taptrust/voucher-protocol/i...,8775634768,None,None,James Levy,2018-12-19 00:00:49+00:00,taptrust/voucher-protocol,NaN,NaN,upgrade to truffle v5
8,ashmastaflash,ashmastaflash,None,CloudPassage,60.0,ash.d.wilson@gmail.com,https://github.com/cloudpassage/jira_halo_issu...,8775652364,None,None,Ash Wilson,2018-12-19 00:05:50+00:00,cloudpassage/jira_halo_issues_sync,NaN,NaN,parameterize concurrency settings
9,qgeissmann,qgeissmann,None,University of British Columbia,60.0,qgeissmann@gmail.com,https://github.com/haneylab/pitally/issues/3,8775693365,None,None,Quentin Geissmann,2018-12-19 00:18:01+00:00,haneylab/pitally,NaN,NaN,make installation pipeline


In [7]:
!open top_1000_github_users.csv

In [8]:
result[result.columns]

,actor_login,best_solver,closed_created_at,company,days_outstanding,email,html_url,issues_id,labels,length_in_days,name,opened_created_at,repo_name,stars,stars_repo_id,title
0,sirpalee,sirpalee,None,@LumaPictures,60.0,sirpalee@gmail.com,https://github.com/LumaPictures/usd-arnold/iss...,8775657555,None,None,Pal Mezei,2018-12-19 00:07:20+00:00,LumaPictures/usd-arnold,11.0,98695067.0,handle aovs and redesign data passed from the ...
1,4rknova,4rknova,None,None,60.0,nikos.papadopoulos@bisimulations.com,https://github.com/4rknova/xtracer/issues/46,8775651419,None,None,Nikos Papadopoulos,2018-12-19 00:05:33+00:00,4rknova/xtracer,9.0,1037499.0,refactor uv integrator
2,KaiKikuchi,KaiKikuchi,None,None,60.0,kai.kikuchi27@gmail.com,https://github.com/KaiKikuchi/ServerRedirect/i...,8775642090,None,None,Kai,2018-12-19 00:02:55+00:00,KaiKikuchi/ServerRedirect,2.0,111564005.0,incompatible with spigot 1.13.2
3,yangwenmai,yangwenmai,None,None,60.0,yangwen.yw@gmail.com,https://github.com/yangwenmai/reading/issues/870,8775679228,None,None,maiyang,2018-12-19 00:13:47+00:00,yangwenmai/reading,1.0,106573721.0,十年磨剑大巧不工
4,MrToph,fiatjaf,None,None,60.0,christoph.michel.1992@gmail.com,https://github.com/ipfs-shipyard/ipscend/issue...,8775669384,None,None,Christoph Michel,2018-12-19 00:10:53+00:00,ipfs-shipyard/ipscend,NaN,NaN,cannot ipfs start
5,birdsarah,mlopatka,None,@mozilla,60.0,sarah@bonvaya.com,https://github.com/mozilla/Overscripted-Data-A...,8775678623,None,None,Sarah Bird,2018-12-19 00:13:36+00:00,mozilla/Overscripted-Data-Analysis-Challenge,NaN,NaN,updating previous fingerprinting / tracking su...
6,philiprbrenan,philiprbrenan,None,Appa Apps Ltd Inc,60.0,philiprbrenan@gmail.com,https://github.com/philiprbrenan/notifications...,8775669647,None,None,philip r brenan,2018-12-19 00:10:58+00:00,philiprbrenan/notifications,NaN,NaN,word to dita plex5 - title and file report com...
7,jamslevy,jamslevy,None,@WeLens,60.0,thelevybreaks@gmail.com,https://github.com/taptrust/voucher-protocol/i...,8775634768,None,None,James Levy,2018-12-19 00:00:49+00:00,taptrust/voucher-protocol,NaN,NaN,upgrade to truffle v5
8,ashmastaflash,ashmastaflash,None,CloudPassage,60.0,ash.d.wilson@gmail.com,https://github.com/cloudpassage/jira_halo_issu...,8775652364,None,None,Ash Wilson,2018-12-19 00:05:50+00:00,cloudpassage/jira_halo_issues_sync,NaN,NaN,parameterize concurrency settings
9,qgeissmann,qgeissmann,None,University of British Columbia,60.0,qgeissmann@gmail.com,https://github.com/haneylab/pitally/issues/3,8775693365,None,None,Quentin Geissmann,2018-12-19 00:18:01+00:00,haneylab/pitally,NaN,NaN,make installation pipeline
